<a href="https://colab.research.google.com/github/MnCSSJ4x/Global-Warming-Detection/blob/main/Time_Series_Analysis_of_Land_Temperature_using_MODIS_DRAFT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Analysis of MODIS gap-filled temperature

In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as nn 
from tensorflow import keras
from sklearn.metrics import mean_absolute_error

In [ ]:
print(tf.__version__)

2.11.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/RE Work/modis_land_temp_full.csv')

In [ ]:
df

,system:index,date,temperature,.geo
0,gf_Day2003_001,2003-01-01,18.668440,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,gf_Day2003_002,2003-01-02,14.234142,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,gf_Day2003_003,2003-01-03,17.625893,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,gf_Day2003_004,2003-01-04,19.030245,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,gf_Day2003_005,2003-01-05,18.106923,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
6565,gf_Day2020_361,2020-12-26,22.429529,"{""type"":""MultiPoint"",""coordinates"":[]}"
6566,gf_Day2020_362,2020-12-27,21.518548,"{""type"":""MultiPoint"",""coordinates"":[]}"
6567,gf_Day2020_363,2020-12-28,20.843983,"{""type"":""MultiPoint"",""coordinates"":[]}"
6568,gf_Day2020_364,2020-12-29,19.812031,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [ ]:
df.drop(labels=['index'],inplace=True,axis="columns")

In [ ]:
df

,system:index,date,temperature,.geo
0,gf_Day2003_001,2003-01-01,18.668440,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,gf_Day2003_002,2003-01-02,14.234142,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,gf_Day2003_003,2003-01-03,17.625893,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,gf_Day2003_004,2003-01-04,19.030245,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,gf_Day2003_005,2003-01-05,18.106923,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
6565,gf_Day2020_361,2020-12-26,22.429529,"{""type"":""MultiPoint"",""coordinates"":[]}"
6566,gf_Day2020_362,2020-12-27,21.518548,"{""type"":""MultiPoint"",""coordinates"":[]}"
6567,gf_Day2020_363,2020-12-28,20.843983,"{""type"":""MultiPoint"",""coordinates"":[]}"
6568,gf_Day2020_364,2020-12-29,19.812031,"{""type"":""MultiPoint"",""coordinates"":[]}"


### Split the dataset into train dates and test dates 

For now we take data till 2014 in train and the rest in test

In [ ]:
split_date = '2015-01-01'

In [ ]:
df_train= df[df['date']<split_date]
df_train

,system:index,date,temperature,.geo
0,gf_Day2003_001,2003-01-01,18.668440,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,gf_Day2003_002,2003-01-02,14.234142,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,gf_Day2003_003,2003-01-03,17.625893,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,gf_Day2003_004,2003-01-04,19.030245,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,gf_Day2003_005,2003-01-05,18.106923,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
4375,gf_Day2014_361,2014-12-27,20.729629,"{""type"":""MultiPoint"",""coordinates"":[]}"
4376,gf_Day2014_362,2014-12-28,18.905086,"{""type"":""MultiPoint"",""coordinates"":[]}"
4377,gf_Day2014_363,2014-12-29,21.893088,"{""type"":""MultiPoint"",""coordinates"":[]}"
4378,gf_Day2014_364,2014-12-30,19.081389,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [ ]:
df_test= df[df['date']>=split_date]
df_test

,system:index,date,temperature,.geo
4380,gf_Day2015_001,2015-01-01,19.138475,"{""type"":""MultiPoint"",""coordinates"":[]}"
4381,gf_Day2015_002,2015-01-02,17.719062,"{""type"":""MultiPoint"",""coordinates"":[]}"
4382,gf_Day2015_003,2015-01-03,17.678950,"{""type"":""MultiPoint"",""coordinates"":[]}"
4383,gf_Day2015_004,2015-01-04,20.483210,"{""type"":""MultiPoint"",""coordinates"":[]}"
4384,gf_Day2015_005,2015-01-05,20.717028,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
6565,gf_Day2020_361,2020-12-26,22.429529,"{""type"":""MultiPoint"",""coordinates"":[]}"
6566,gf_Day2020_362,2020-12-27,21.518548,"{""type"":""MultiPoint"",""coordinates"":[]}"
6567,gf_Day2020_363,2020-12-28,20.843983,"{""type"":""MultiPoint"",""coordinates"":[]}"
6568,gf_Day2020_364,2020-12-29,19.812031,"{""type"":""MultiPoint"",""coordinates"":[]}"


### Sliding Window

In [ ]:
def get_time_series_data(sequence,n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
 # find the end of this pattern
    end_ix = i + n_steps
 # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
 # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)

  return np.array(X), np.array(y)


In [ ]:
n_steps = 3
n_features = 1

In [ ]:
# X,y = get_time_series_data(df_train['temperature'].to_numpy(),3)

In [ ]:
# X

In [ ]:
# y

In [ ]:
# X = X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
# X[0]

In [ ]:
# X.shape

### LSTM model 

In [ ]:
# define model
model = Sequential()
model.add(nn.LSTM(200, activation='relu', input_shape=(n_steps, n_features)))
model.add(nn.Dense(1))
opt = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=opt, loss='mae')

In [ ]:
# model.fit(X, y, epochs=100, verbose=1)

### Save Model 

In [ ]:
# tf.saved_model.save(model, 'drive/MyDrive/RE Work/')

### Test Evaluation 

In [ ]:
# X_test,y_test = get_time_series_data(df_test['temperature'].to_numpy(),3)

In [ ]:
# y_pred = model.predict(X_test)

In [ ]:
# mae = mean_absolute_error(y_test, y_pred)
# print('MAE:', mae)

### Function to train and test on various window sizes

In [ ]:
def train_test(df_train,df_test,n_steps,n_features,model,epochs=100):
  '''
    does the training as well as training and returns MAE on test data 
    
    @input: df_train: Training Dataset, df_test: Testing Dataset, 
            n: window size for time series, model: A compiled tensorflow model object
    @output: mae 
  '''
  
  X_train,y_train = get_time_series_data(df_train['temperature'].to_numpy(),n_steps)
  X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
 
  model.fit(X_train, y_train, epochs=epochs, verbose=1)
  
  X_test,y_test = get_time_series_data(df_test['temperature'].to_numpy(),n_steps)
  y_pred = model.predict(X_test)
  mae = mean_absolute_error(y_test, y_pred)
  return mae 

In [ ]:
# mae = train_test(df_train,df_test,n_steps,n_features,model)
# mae

In [ ]:
vals = [3,7,15,30]
mae = []
for i in vals: 
  print("For window size: ",i)
  mae_val =  train_test(df_train,df_test,i,n_features,model)
  mae.append(mae_val)
df_mae = pd.concat([pd.DataFrame(vals,columns=["Window Size"]),pd.DataFrame(mae,columns=["Mean Absolute Error"])],axis=1)

For window size:  3
Epoch 1/100
137/137 [==============================] - 1s 8ms/step - loss: 2.5682
Epoch 2/100
137/137 [==============================] - 1s 8ms/step - loss: 2.6065
Epoch 3/100
137/137 [==============================] - 1s 8ms/step - loss: 2.5664
Epoch 4/100
137/137 [==============================] - 1s 8ms/step - loss: 2.5727
Epoch 5/100
137/137 [==============================] - 1s 8ms/step - loss: 2.5429
Epoch 6/100
137/137 [==============================] - 1s 10ms/step - loss: 2.5318
Epoch 7/100
137/137 [==============================] - 2s 16ms/step - loss: 2.5278
Epoch 8/100
137/137 [==============================] - 2s 11ms/step - loss: 2.5366
Epoch 9/100
137/137 [==============================] - 1s 8ms/step - loss: 2.5921
Epoch 10/100
137/137 [==============================] - 1s 8ms/step - loss: 2.5547
Epoch 11/100
137/137 [==============================] - 1s 8ms/step - loss: 2.5409
Epoch 12/100
137/137 [==============================] - 1s 8ms/step - los

In [ ]:
df_mae

,Window Size,Mean Absolute Error
0,3,2.396893
1,7,2.289131
2,15,2.315980
3,30,2.564344
